In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(37) 

In [2]:
# 寻找相似用户，此处使用Pearson 相关性来确定相似度
# 将FireAI_032中的函数导入进来，
# 定义一个函数来计算两个用户之间的皮尔逊相关系数
def pearson_score(dataset,user1,user2):
    # 和上面的函数类似，先排除相似度为0的情况
#     both_rated_num=0 # 表示同时被两个用户都评价过的电影的数目
#     for item in dataset[user1]: # 在user1评价过的电影中
#         if item in dataset[user2]: # 如果user2也评价过，则+1
#             both_rated_num+=1
#     if both_rated_num==0:# 不存在同时被两个用户都评价过的电影
#         return 0 # 直接返回0，表示两个用户之间相似度为0

    both_rated={}
    for item in dataset[user1]:
        if item in dataset[user2]:
            both_rated[item]=1
    num_ratings=len(both_rated)
    if num_ratings==0: # 不存在同时被两个用户都评价过的电影
        return 0
    
    # 计算每个用户对每个相同电影的评价之和
    user1_sum=np.sum([dataset[user1][item] for item in both_rated])
    user2_sum=np.sum([dataset[user2][item] for item in both_rated])
    
    # 计算每个用户对每个相同电影的评价的平方和
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in both_rated])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in both_rated])

    # 计算两个用户的相同电影的乘积
    product_sum=np.sum([dataset[user1][item]*dataset[user2][item] for item in both_rated])
    
    # 计算Pearson 相关系数
    Sxy = product_sum - (user1_sum * user2_sum / num_ratings)
    Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
    Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
    
    if Sxx * Syy == 0:
        return 0

    return Sxy / np.sqrt(Sxx * Syy)


In [3]:
def find_similar_users(dataset, user, user_num=3):
    if user not in dataset: # 首先保证user在dataset中
        raise TypeError('User {} not in dataset!'.format(user))
    
    # 对于所有用户，计算其与user的相似度，此处使用Pearson相关性
    scores=np.array([[other_user,pearson_score(dataset,user,other_user)] for 
                         other_user in dataset if other_user!=user])
    # 相关性都存放在scores这个二维矩阵中，故而可以通过排序来寻找相似用户
    scores_sorted=np.argsort(scores[:,1])[::-1] # 先排序取坐标，再逆序
    # 获取最前面的user_num个相似用户
    top_users=scores_sorted[:user_num]
    return scores[top_users] # 返回这些user的信息


In [4]:
# 使用电影数据来寻找相似用户
import json
with open("E:\PyProjects\DataSet\FireAI\movie_ratings.json",'r') as file:
    dataset=json.loads(file.read())

user='John Carson'
similar_users=find_similar_users(dataset, user, 4)
print('Users similar to {}---->>>'.format(user))
print('User\t\t\tSimilarity Score\n')
for item in similar_users:
    print('{}\t\t{}'.format(item[0],round(float(item[1]),3)))

Users similar to John Carson---->>>
User			Similarity Score

Michael Henry		0.991
Alex Roberts		0.747
Melissa Jones		0.594
Jillian Hobart		0.567


In [5]:
# 创建电影推荐引擎
def get_recommendations(dataset,user):
    if user not in dataset: # 首先保证user在dataset中
        raise TypeError('User {} not in dataset!'.format(user))
    
    total_scores={} # 存放的key为电影名称，value为对该电影的评价乘以相似度
    similarity_sums={} # 存放的key为电影名称，value为相似度
    for other_user in dataset:
        if other_user ==user: continue  # 确保是其他用户而非自身
        similarity_score=pearson_score(dataset,user, other_user) 
#         print('other user: ', other_user, 'similarity: ', similarity_score)
        if similarity_score<=0: continue # 如果相似度太小则忽略
        
        # 找到还未被该用户评分,但是却被other_user评分过的电影
        # 这部分电影表示相似的other_ser已经看过但是user没有看过的电影，
        # 推荐的电影肯定来自于这部分
        user_not_rating_movies=[]
        for movie in dataset[other_user]: # other_user评分过的电影
#             if movie not in dataset[user] or dataset[user][movie]==0: 
            if movie not in dataset[user]: 

                # 但是user没有评分过，或者user评分为0(是不是系统认为没有评分那评分就是0）)
                user_not_rating_movies.append(movie)
#         print(user_not_rating_movies)
        # 计算这些user没有评分过的电影的推荐分数，
        # 推荐分数此处用该电影的评价乘以相似度表示
        for movie in user_not_rating_movies:
            recommend_score=dataset[other_user][movie]*similarity_score
            total_scores.update({movie: recommend_score})
            similarity_sums.update({movie: similarity_score})
#         print('other user: ', other_user, 'total_scores: ', total_scores)
    # 判断total_scores，如果推荐的总数为0，表示所有电影都被user评价过，则不推荐
    if len(total_scores) ==0: return [[0,'No Recommendations']]
    
    # 计算每个电影的推荐等级
    movie_ranks=np.array([[rec_score/similarity_sums[movie],movie] for 
                         movie, rec_score in total_scores.items()])
    # 对第一列进行逆序排列
    movie_ranks_desc=movie_ranks[np.argsort(movie_ranks[:,0])[::-1]]
#     print(movie_ranks_desc)
    return movie_ranks_desc

In [6]:
import json
with open("E:\PyProjects\DataSet\FireAI\movie_ratings.json",'r') as file:
    dataset=json.loads(file.read())

user='John Carson'
movie_ranks=get_recommendations(dataset,user)
print('Recommended movies to {}---->>>'.format(user))
for idx, recommend in enumerate(movie_ranks):
    print('{}: {}-->recommend score: {}'.format(idx, recommend[1], recommend[0]))

user='Michael Henry'
movie_ranks=get_recommendations(dataset,user)
print('Recommended movies to {}---->>>'.format(user))
for idx, recommend in enumerate(movie_ranks):
    print('{}: {}-->recommend score: {}'.format(idx, recommend[1], recommend[0]))


Recommended movies to John Carson---->>>
0: No Recommendations-->recommend score: 0
Recommended movies to Michael Henry---->>>
0: Jerry Maguire-->recommend score: 3.0
1: Inception-->recommend score: 3.0
2: Anger Management-->recommend score: 2.0
